In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import random

import numpy as np
import matplotlib.pyplot as plt
import math
import random

# Create chromosome blueprint, initialize the population, and create a way to display
# the population

class Chromosome:
    
    def __init__(self, genes = None):
        
        if(genes == None):
            x = np.random.randint(-100, 100)
            y = np.random.randint(-100, 100)
            
        else:
            x = genes["x"]
            y = genes["y"]
    
        #self.genes = {"x": x}
        self.genes = {"x": x, "y": y}
        
        self.fitness = None
        
def create_population(num_chromosomes):
    
    population = []
    for i in range(num_chromosomes):
        population.append(Chromosome())
    return population 

def display_population(population, string):
    
    print(f"\nPopulation after {string}:\n")
    for i, chromosome in enumerate(population):
        print("Chromosome %s : x = %s, y = %s, fitness = %s" 
              % (i, chromosome.genes["x"], chromosome.genes["y"], chromosome.fitness))



In [2]:
# Create fitness function, f(x) = x^2
# maps a chromosome representation into a scalar value

def fitness(x, y):
    
    return (x ** 2 + y ** 2) * 0.26 - (x * y) * 0.48

# Create function to update each chromosome's fitness

def evaluate_population(population):

    for chromosome in population:
        chromosome.fitness = fitness(chromosome.genes["x"], chromosomes.genes["y"])
        
    scores = [chromosome.fitness for chromosome in population]
    
    indices = np.argsort(scores) #this sorts each row from greatest to least and creates a list of indices
    
    return list(np.asarray(population)[indices])

In [8]:
# Create selection function(s). this is the function that chooses parents to reproduce.

# generation_gap defaults to 0, will only be populated if elitism is true

def roulette_wheel(population, p_num_selected):
    
    sum_of_fitnesses = calculate_sum_of_fitnesses(population)
    
    # calculate relative fitness. Roulet wheel made
    
    try:
        scores = [(1 - (chromo.fitness / sum_of_fitnesses)) for chromo in population]
    except:
        print("LOOK AT ME: " sum_of_fitness)
        exit()
    # spin the created roulet wheel --> P selected chromos
    
    selected = []
    
    for i in range(p_num_selected):
        
        sigma = random.uniform(0,1)
    
        total = 0
        for j in range(len(scores)):
            total = total + scores[j]
            if(total >= sigma):
                break
        
        selected.append(population[j])
   
    return selected 

def calculate_sum_of_fitnesses(population):
    sum_of_fitnesses = 0
    for chromosome in population:
        sum_of_fitnesses += chromosome.fitness
    return sum_of_fitnesses

def create_matches(parent_group, req_num_parents):
    
    num_groups = int(len(parent_group) / req_num_parents)
    
    # get sets of subgroups that are going to produce one child
    
    couples = []
        
    for i in range(num_groups):
        
        # first get the indices of parents for each subgroup
        
        indices = [ np.random.randint(0, len(parent_group)) 
                    for j in range(req_num_parents) ]
        
        # populate subgroup with chromosomes 
        
        group = [parent_group[index] for index in indices]
        
        couples.append(group)
    
    return couples

def selection(population, elite_percent, req_num_parents = 2):
    
    parents = []
    
    if elite_percent > 0:
        
        elite_group = []
        
        num_chromosomes = len(population)
        e_num_selected = math.ceil(elite_percent * num_chromosomes) 
        
        # most elite chromos automatically get into next generation
        
        for i in range(e_num_selected):
            elite_group.append(population[i])
    
    else:
        elite_group = []
        e_num_selected = 0
        
    # let's build up the rest of the parents list using roulette wheel.
   
    num_chromosomes = len(population)
    p_num_selected = int((num_chromosomes - e_num_selected) * req_num_parents)
    
    parent_group = roulette_wheel(population, p_num_selected)
    
    parent_group = create_matches(parent_group, req_num_parents)

    return elite_group, parent_group

In [9]:
# Create simplex cross over function(s)
# you can choose to have two parents, three parents, k parents
# if you do k-point, stick with two parents
###### i really don't know what kind of crossover i did here ####

def calculate_mean(population):
    
    num_genes = len(population[-1].genes.keys())
    
    mean = np.zeros(num_genes)
    for chromosome in population:
        genes = chromosome.genes
        for i, current_gene in enumerate(genes.keys()):
            mean[i] += chromosome.genes[current_gene]
        
    return mean / len(population)
        
def simplex_crossover(parent_groups):
    
    children = []
    for parents in parent_groups:
    
        # select random parent for simplex equation 
        
        index = np.random.randint(0, len(parents))
        rand_chromo = parents[index]
        
        # get all possible genes
        
        all_genes = parents[-1].genes.keys()
        
        # run simplex on each gene
        
        simplexed_genes = {}
        for current_gene in all_genes:
            
            epsilon = np.random.rand()
            info = [chromo.genes[current_gene] for chromo in parents] 
            agg = math.ceil(np.mean(info) + (rand_chromo.genes[current_gene] - np.mean(info)) * epsilon)
            
            simplexed_genes[current_gene] = agg
        
        # create child from simplexed genes
        
        children.append(Chromosome(simplexed_genes))
    
    return children

In [10]:
# implement mutation here

def mutation(children, mutate_chance, mutate_scale = 0.1):
    
    kiddos = []
    
    for child in children:
        
        alpha = np.random.rand()
        
        # mutate if random allows 
        
        if(alpha <= mutate_chance):
            
            # mutate all child genes
            
            all_genes = child.genes.keys()
        
            for current_gene in all_genes:
                flip = -1 if(np.random.rand() >= 0.5) else 1
                offset = flip * child.genes[current_gene] * mutate_scale
                child.genes[current_gene] = math.ceil(child.genes[current_gene] + offset)
            
        kiddos.append(child)
    
    return kiddos 

def mutation_constant(population, generation):
    
    # the strategy is to decrease sigma as the number of generations increases so we get 
    # small variations near the optimum, preventing individuals from jumping over the 
    # minimum
    
    if generation >= num_generations * (3/4):
            sigma = 0.05
    elif generation < num_generations * (3/4) and generation >= num_generations / 2:
            sigma = 0.1
    elif generation < num_generations / 2 and generation >= num_generations / 4:
            sigma = 0.15
    elif generation < num_generations / 4:
            sigma = 0.2
            
    # each individual is mutated by adding a gaussian random value
    gaussian = random.uniform(0,sigma)
    # can I make this value hover around zero?
    #gaussian = random.uniform(-sigma,sigma)
    
    for chromosome in population:
        chromosome.genes["x"] = chromosome.genes["x"] + gaussian
        
    # evaluate fitness before returning
    evaluate_population(population)
    
    return population

In [11]:
# stats methods
def get_min(population):
    
    fitness = []
    for chromosome in population:
        fitness.append(chromosome.fitness)
    return min(fitness)
    
def get_max(population):
    fitness = []
    for chromosome in population:
        fitness.append(chromosome.fitness)
    return max(fitness)

def get_mean(population):
    fitness = []
    sum = 0
    for chromosome in population:
        sum += chromosome.fitness
    return sum / len(population)

In [15]:
# this is the main function #
    
num_chromosomes = 20
num_generations = 100
elite_percent = 0.0
mutate_chance = 0.25
display_target = 50

fitness_min = []
fitness_max = []
fitness_avg = []

# Create population

pop = create_population(num_chromosomes)

generation = 1

for i in range(num_generations):
    
    if(i % display_target == 0):
        nl = "\n"
        print(f"{nl}Generation {generation}{nl}")
    
    # Calcuate fitness (the environment scored each chromosome) and update analytics

    pop = evaluate_population(pop)
    
    if(i % display_target == 0):
        if(i == 0):
            display_population(pop, "Initial")
        else:
            display_population(pop, "Updated")
        
    fitness_min.append(get_min(pop))
    fitness_max.append(get_max(pop))
    fitness_avg.append(get_mean(pop))
    
    # Selection for children and optional elitism 
    
    elite_group, parent_group = selection(pop, elite_percent)
    
    # Create some kids, via cross-over / mutation 
    
    children = simplex_crossover(parent_group)
    
    children = mutation(children, mutate_chance)
    
    #display_population(children,"mutation")
    
    # Update population for new generation 
    
    pop = elite_group + children
    
    if(i % display_target == 0):
        display_population(elite_group, "elitism")
    
    # Update generations 
    
    generation +=1
    
# Plot Results

plt.plot(fitness_min,'r', label = 'min')
plt.plot(fitness_avg,'b', label = 'average')
plt.plot(fitness_max,'g', label = 'max')
plt.ylabel('fitness')
plt.xlabel('generations')
plt.legend(loc='upper right')
plt.show()


Generation 1


Population after Initial:

Chromosome 0 : x = -5, fitness = 25
Chromosome 1 : x = 7, fitness = 49
Chromosome 2 : x = 13, fitness = 169
Chromosome 3 : x = 17, fitness = 289
Chromosome 4 : x = 19, fitness = 361
Chromosome 5 : x = -20, fitness = 400
Chromosome 6 : x = 22, fitness = 484
Chromosome 7 : x = 25, fitness = 625
Chromosome 8 : x = -31, fitness = 961
Chromosome 9 : x = -33, fitness = 1089
Chromosome 10 : x = -46, fitness = 2116
Chromosome 11 : x = -55, fitness = 3025
Chromosome 12 : x = 60, fitness = 3600
Chromosome 13 : x = -61, fitness = 3721
Chromosome 14 : x = -63, fitness = 3969
Chromosome 15 : x = 68, fitness = 4624
Chromosome 16 : x = 71, fitness = 5041
Chromosome 17 : x = -89, fitness = 7921
Chromosome 18 : x = -96, fitness = 9216
Chromosome 19 : x = -100, fitness = 10000

Population after elitism:



ZeroDivisionError: float division by zero

In [16]:
# do a 3d visualization of this space